In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-09-10 16:21:06--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-09-10 16:21:07--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc77b038a1ed65a47cec5856c31.dl.dropboxusercontent.com/cd/0/get/BstMsxWogwbdswxlZT2gIFeD949vH15LKDiE5yQOpxSm9iLDNYXO5e6FOrKxrvNKvOeIBKktbgqJ5NXW7_QKcalzcGTUDsNFYeaVZft1caTCVeaFLAKy2kzXdYRgTlJGqfNJrInAYzzEaCW_TH0knKQMH9mngWqrzyT55CtT51zfQOmJYAC8NOucaZhF-8LdpbE/file?dl=1# [following]
--2022-09-10 16:21:07--  https://ucc77b038a1ed65a47cec5856c31.dl.dropboxusercontent.com/cd/0/get/BstMsxWogw

In [ ]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab

## Download the data


In [ ]:
mecab = Mecab()

In [ ]:
!gdown --id 1V6HsBoEczDoo4NDZ1I5iXSfRxFxCatis
!gdown --id 1E8bJN1IdXtLzsLysE4HHRf_bTPXpOdBX

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1V6HsBoEczDoo4NDZ1I5iXSfRxFxCatis
To: /content/1_구어체(1).xlsx
100% 15.8M/15.8M [00:00<00:00, 170MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1E8bJN1IdXtLzsLysE4HHRf_bTPXpOdBX
To: /content/1_구어체(2).xlsx
100% 14.3M/14.3M [00:00<00:00, 99.0MB/s]


In [ ]:
import pandas as pd

df = pd.read_excel('/content/1_구어체(1).xlsx')
df
df2 = pd.read_excel('/content/1_구어체(2).xlsx')
df2

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
199995,399996.0,당신은 곧 당신의 조카를 볼 수 있어요.,You can see your nephew soon.
199996,399997.0,나는 당신이 내게 공유한 음악을 들었어요.,I've listened to the music that you shared wit...
199997,399998.0,그 서류는 당신이 공유해 준 것과 달라요.,The document is different from what you shared.
199998,399999.0,당신이 잡지에 광고 낸 것을 봤어요.,I saw your advertisement in the magazine.


In [ ]:
src_ls = df['원문'].to_list()
len(src_ls)

200000

In [ ]:
trg_ls = df['번역문'].to_list()
len(trg_ls)

200000

In [ ]:
src_ls2 = df2['원문'].to_list()
len(src_ls2)

200000

In [ ]:
trg_ls2 = df2['번역문'].to_list()
len(trg_ls2)

200000

In [ ]:
src_ls.extend(src_ls2)
trg_ls.extend(trg_ls2)

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
hidden_units = 256  # Latent dimensionality of the encoding space.
embedding_dim = 32
num_samples = 400000  # Number of samples to train on.
dropout_rate = 0.3

In [ ]:
encoder_input = []
for src in src_ls:
  encoder_input.append(mecab.morphs(src))

len(encoder_input)

400000

In [ ]:
decoder_input = []
decoder_target = []

for trg in trg_ls:
  trg_line_in = [w for w in ("<sos> " + trg).split()]
  trg_line_out = [w for w in (trg + " <eos>").split()]  
  decoder_input.append(trg_line_in)
  decoder_target.append(trg_line_out)

In [ ]:
len(decoder_input)

400000

In [ ]:
len(decoder_target)

400000

In [ ]:
# 한국어
tokenizer_kor = Tokenizer(filters="", lower=False)
tokenizer_kor.fit_on_texts(encoder_input)

# 인코더 데이터
encoder_input = tokenizer_kor.texts_to_sequences(encoder_input)

In [ ]:
# 영어
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(decoder_input)
tokenizer_en.fit_on_texts(decoder_target)

# 디코더 데이터
decoder_input = tokenizer_en.texts_to_sequences(decoder_input)
decoder_target = tokenizer_en.texts_to_sequences(decoder_target)

In [ ]:
decoder_target[-5:]

[[47, 26, 103, 23, 6913, 555, 3],
 [181, 3189, 5, 1, 511, 13, 7, 2366, 14, 95, 3],
 [19, 496, 8, 203, 34, 78, 7, 14430, 3],
 [4, 374, 23, 2361, 11, 1, 10666, 3],
 [37, 135, 733, 384, 66, 159, 150, 3]]

In [ ]:
encoder_input = pad_sequences(encoder_input, padding='post')
decoder_input = pad_sequences(decoder_input, padding='post')
decoder_target = pad_sequences(decoder_target, padding='post')

In [ ]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (400000, 62)
디코더의 입력의 크기(shape) : (400000, 54)
디코더의 레이블의 크기(shape) : (400000, 54)


In [ ]:
src_vocab_size = len(tokenizer_kor.word_index) + 1
tar_vocab_size = len(tokenizer_en.word_index) + 1

print(f"영어 단어 집합의 크기 : {tar_vocab_size}, 한국어 단어 집합의 크기 : {src_vocab_size}")

영어 단어 집합의 크기 : 124036, 한국어 단어 집합의 크기 : 61697


In [ ]:
src_to_index = tokenizer_kor.word_index  # word : idx
index_to_src = tokenizer_kor.index_word  # idx : word
tar_to_index = tokenizer_en.word_index # word : idx
index_to_tar = tokenizer_en.index_word # idx : word

In [ ]:
print(src_to_index)
print(index_to_src)

{'.': 1, '는': 2, '이': 3, '을': 4, '에': 5, '하': 6, '은': 7, '가': 8, '의': 9, '있': 10, '를': 11, '습니다': 12, '고': 13, '나': 14, '?': 15, ',': 16, '한': 17, '들': 18, '어요': 19, '것': 20, '당신': 21, '그': 22, '어': 23, '에서': 24, '수': 25, '지': 26, '주': 27, '저': 28, '으로': 29, '입니다': 30, '게': 31, '도': 32, '우리': 33, '할': 34, '로': 35, '기': 36, '해': 37, '었': 38, '되': 39, '합니다': 40, '했': 41, '과': 42, '내': 43, '않': 44, '에게': 45, '적': 46, '좋': 47, '일': 48, '와': 49, '면': 50, '겠': 51, '해요': 52, '없': 53, '제': 54, '아': 55, '세요': 56, '시': 57, '사람': 58, '에요': 59, '다': 60, '거': 61, '보': 62, '싶': 63, '같': 64, '말': 65, '생각': 66, '인': 67, '나요': 68, '너': 69, '았': 70, '때': 71, '받': 72, '만': 73, '더': 74, '야': 75, '많': 76, '친구': 77, '잘': 78, '한국': 79, '오늘': 80, '안': 81, '시간': 82, '지만': 83, '때문': 84, '보내': 85, '된': 86, '중': 87, '네': 88, '제품': 89, '번': 90, '까지': 91, '사용': 92, '지금': 93, '아요': 94, '그녀': 95, '그것': 96, '알': 97, '먹': 98, '해서': 99, '여': 100, '해야': 101, '전': 102, '집': 103, '서': 104, '다고': 105, '필요': 106, '가지': 107, 

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [193823 349622   8082 ... 256076 228663 208009]


In [ ]:
# 배열
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
decoder_input[indices[0]], decoder_target[indices[0]]

(array([     2,    104,      7,   2395,    332,     39, 107038,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0], dtype=int32),
 array([   104,      7,   2395,    332,     39, 107038,      3,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0], dtype=in

In [ ]:
n_of_val = int(num_samples*0.1)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (360000, 62)
훈련 target 데이터의 크기 : (360000, 54)
훈련 target 레이블의 크기 : (360000, 54)
테스트 source 데이터의 크기 : (40000, 62)
테스트 target 데이터의 크기 : (40000, 54)
테스트 target 레이블의 크기 : (40000, 54)


In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Bidirectional, Concatenate
from tensorflow.keras.models import Model

In [ ]:
hidden_units2 = hidden_units * 2

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)# 임베딩 층
#initializer = tf.keras.initializers.RandomUniform(minval=-0.8, maxval=0.8)
encoder_lstm = Bidirectional(LSTM(hidden_units, return_sequences = True, return_state=True,dropout=0.3))
lstm1 = Bidirectional(LSTM(hidden_units, return_sequences = True, return_state=True,dropout=0.3))

x = encoder_lstm(enc_emb)
x = lstm1(x)
x = lstm1(x)

encoder_outputs, forward_h, forward_c, backward_h, backward_c = lstm1(x) # 은닉 상태와 셀 상태를 리턴

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

print(state_h.shape,state_c.shape,encoder_outputs.shape)
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

(None, 512) (None, 512) (None, None, 512)


In [ ]:
from tensorflow.keras.layers import Attention
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 임베딩 결과

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units2, return_sequences=True, return_state=True,dropout=0.3) 
de_lstm1 = LSTM(hidden_units2, return_sequences = True, return_state=True,dropout=0.3)


x = decoder_lstm(dec_emb)
x = de_lstm1(x)
x = de_lstm1(x)
# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용

decoder_outputs,_,_ = de_lstm1(x)
# state_h[:, tf.newaxis, :].shape
# attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1) # query 만들기

attention = Attention(hidden_units)
context_vector, _ = attention([ S_, encoder_outputs], return_attention_scores = True) # '_' 자리는 score 위치     encoder_outputs : key 와 value 

concat = tf.concat([decoder_outputs, context_vector], axis=-1) # S햇 완성

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(concat)

In [ ]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)# 임베딩 층
x = Bidirectional(LSTM(hidden_units, return_sequences = True, return_state=True, dropout = dropout_rate))(enc_emb)
encoder_lstm = Bidirectional(LSTM(hidden_units, return_sequences = True, return_state=True, dropout = dropout_rate)) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(x) # 은닉 상태와 셀 상태를 리턴

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [ ]:
from tensorflow.keras.layers import Attention
# 디코더
#hidden_units = hidden_units*4

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 임베딩 결과

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = Bidirectional(LSTM(hidden_units2, return_sequences=True, return_state=True, dropout = dropout_rate))

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
x = decoder_lstm(dec_emb, initial_state = encoder_states)
decoder_outputs,_,_,_,_ = Bidirectional(LSTM(hidden_units*2, return_sequences=True, return_state=True, dropout = dropout_rate))(x)

# attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs[:, :-1, :]], axis=1) # query 만들기

attention = Attention(hidden_units2)
context_vector, _ = attention([ S_, encoder_outputs], return_attention_scores = True) # '_' 자리는 score 위치     encoder_outputs : key 와 value 

concat = tf.concat([decoder_outputs, context_vector], axis=-1) # S햇 완성

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(concat)

In [ ]:
# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 32)     1974304     ['input_2[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 256)    31753216    ['input_3[0][0]']                
                                                                                              

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png' )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, callbacks=[es],
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=64, epochs=50)

Epoch 1/50
5625/5625 [==============================] - 2361s 416ms/step - loss: 1.0729 - acc: 0.8451 - val_loss: 0.7508 - val_acc: 0.8823
Epoch 2/50
5625/5625 [==============================] - 2344s 417ms/step - loss: 0.6351 - acc: 0.8961 - val_loss: 0.4941 - val_acc: 0.9208
Epoch 3/50
5625/5625 [==============================] - 2346s 417ms/step - loss: 0.4257 - acc: 0.9231 - val_loss: 0.3756 - val_acc: 0.9391
Epoch 4/50
5625/5625 [==============================] - 2347s 417ms/step - loss: 0.3000 - acc: 0.9387 - val_loss: 0.3252 - val_acc: 0.9477
Epoch 5/50
5625/5625 [==============================] - 2347s 417ms/step - loss: 0.2129 - acc: 0.9516 - val_loss: 0.2990 - val_acc: 0.9534
Epoch 6/50
5625/5625 [==============================] - 2348s 417ms/step - loss: 0.1530 - acc: 0.9630 - val_loss: 0.2813 - val_acc: 0.9573
Epoch 7/50
5625/5625 [==============================] - 2348s 417ms/step - loss: 0.1177 - acc: 0.9702 - val_loss: 0.2705 - val_acc: 0.9600
Epoch 8/50
5625/5625 [=====

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('/content/drive/MyDrive/trans_koreng_project/모델 결과저장/bilstm_attention_4layer_dropout0.3_emb_dim32_model.h5')

In [ ]:
# 인코더 - 훈련과 동일 
encoder_model = Model(encoder_inputs, [encoder_outputs, encoder_states])

In [ ]:
# 디코더 

# 이전 시점의 상태를 보관할 텐서 자리 만들어 주기
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

encoder_state_h = Input(shape=(hidden_units,))
encoder_outputs2 = Input(shape =(None, hidden_units,))

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h, state_c = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h, state_c]

# Attention
S_ = tf.concat([state_h[:, tf.newaxis, :], decoder_outputs2[:, :-1, :]], axis=1) # query 

context_vector, att_score = attention([decoder_outputs2, encoder_outputs2], return_attention_scores = True)
decoder_concat = tf.concat([S_, context_vector], axis=-1)

# 모든 시점에 대해서 단어 예측 (Fully Connected)
decoder_outputs2 = decoder_dense(decoder_concat)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs, encoder_state_h, encoder_outputs2] + decoder_states_inputs,
    [decoder_outputs2, att_score] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : 내 가 만약 초능력 을 가지 게 된다면 순간이동 을 하고 싶다 . 
정답문장 : I want to teleport if I have superpowers . 
번역문장 : I want to be a good friend in Korea . 
--------------------------------------------------
입력문장 : 나 는 보통 게으른 편입 니 다 . 
정답문장 : I m generally lazy . 
번역문장 : I m a little bit tired . 
--------------------------------------------------
입력문장 : 당신 은 어느 팀 이 이길 것 으로 예측 하십니까 
정답문장 : Which team do you think will win ? 
번역문장 : Do you have any other baggages to check the w
--------------------------------------------------
입력문장 : 나 는 팬미팅 을 마치 고 독립 기념관 에 다녀왔습니다 . 
정답문장 : I went to the Independece Hall after finishing the fan meeting . 
번역문장 : I went to the library after finishing my work 
--------------------------------------------------
입력문장 : 더불어서 호텔 정보 와 호텔 에서 사무실 가는 방법 알려주기 바랍니다 . 
정답문장 : Besides please tell me how to get to the office from the hotel . 
번역문장 : And I would like to know if the product arriv
--------------------------------------------------


In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print("-"*50)

입력문장 : 그러나 대학교 교육 은 자기 주도 적 인 학습 을 하도록 했어요 . 
정답문장 : But university education made me self study . 
번역문장 : But the first time in the world was to be a p
--------------------------------------------------
입력문장 : 이 것 은 현재 거의 찾아볼수 없고 효과 도 미미했어요 . 
정답문장 : It s hard being seen these days and its effects are rather lackluster . 
번역문장 : This is a problem with the poor people in the
--------------------------------------------------
입력문장 : 업무 진행 이 되도록 확인 좀 부탁드려요 . 
정답문장 : Please check it out so that we can progress the tasks . 
번역문장 : Please check the schedule of the new one . 
--------------------------------------------------
입력문장 : 이러한 관점 에서 프랜디 와 키덜트 의 관계 가 주목 할 만해 요 . 
정답문장 : From this perspective the relationship between friendy and kidult is worth attention . 
번역문장 : The reason why I like this is that the world is a
--------------------------------------------------
입력문장 : 나 는 네 시 에 저절로 잠 에서 깼어요 . 
정답문장 : I woke up at four automatically . 
번역문장 : I was so tired . 
----------------------

## BLEU Score (Bilingual Evaluation Understudy Score)
기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법 중 하나인 BLEU(Bilingual Evaluation Understudy)를 측정해봅시다. 
- 논문 : BLEU: a Method for Automatic Evaluation of Machine Translation
- BLEU 점수는 기계 번역된 텍스트와 고품질 참조 번역 세트의 유사성을 측정하는 0과 1 사이의 숫자입니다. 
    - 값이 0이면 기계 번역된 출력이 참조 번역과 겹치는 부분이 없는 것을 의미하고(저품질)
    - 1이면 참조 번역과 완벽하게 겹치는 것을 의미합니다(고품질)

- BLEU 설명 
    - https://jrc-park.tistory.com/273
    - https://wikidocs.net/31695
    - https://cloud.google.com/translate/automl/docs/evaluate?hl=ko#bleu

In [ ]:
import numpy as np
from collections import Counter
from nltk import ngrams

BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 

In [ ]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

패키지 NLTK의 BLEU : 0.5045666840058485


In [ ]:
ref_train = [seq_to_tar(sen) for sen in decoder_input_train[:100]]

In [ ]:
from tqdm.notebook import tqdm 

pred_train = []
for idx in tqdm(range(len(encoder_input_train[:100]))):
    pred_train.append(decode_sequence(encoder_input_train[idx:idx+1]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bleu_score = 0
for ref, pred in zip(ref_train, pred_train):
    if len(ref) == 0 or len(pred) == 0:
        continue
    bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_train)
bleu_score

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

1.3520424645553526e-231

In [ ]:
ref_test = [seq_to_tar(sen) for sen in decoder_input_test[:100]]
pred_test = [decode_sequence(encoder_input_test[idx:idx+1]) for idx in tqdm(range(len(encoder_input_test[:100])))]

bleu_score = 0
for ref, pred in zip(ref_test, pred_test):
    if len(ref) == 0 or len(pred) == 0:
        continue
    bleu_score += bleu.sentence_bleu(ref, pred)

bleu_score = bleu_score/len(ref_test)
bleu_score

  0%|          | 0/100 [00:00<?, ?it/s]

1.3361875311866586e-231

### Reference

- https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
- https://wikidocs.net/86900